# Q3

In [1]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv("../HW3_train.csv")
df = df[['X3', 'X4', 'price']].dropna(subset=['X3', 'X4'])
df.shape

(10700, 3)

In [42]:
def relu(x):
    return max(0, x)

In [43]:
X = df[['X3', 'X4']].values
Y = df['price'].values

In [50]:
w = np.random.randn(2, 2)   # w_2,j,i
v = np.random.randn(2)      # w_1,j

In [51]:
lr = 0.01
epochs = 100

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(X)):
        x1, x2 = X[i]
        target = Y[i]

        h1 = relu(w[0,0] * x1 + w[0,1] * x2)
        h2 = relu(w[1,0] * x1 + w[1,1] * x2)
        f_x = v[0] * h1 + v[1] * h2

        loss = (target - f_x)**2
        total_loss += loss

        grad_f_x = -2 * (target - f_x)

        grad_v = np.arraygrad_v = np.array([grad_f_x * h1, grad_f_x * h2])
        grad_h = np.array([grad_f_x * v[0], grad_f_x * v[1]])
        
        grad_w = np.zeros_like(w)
        grad_w[0] = grad_h[0] * (h1 > 0) * np.array([x1, x2])
        grad_w[1] = grad_h[1] * (h2 > 0) * np.array([x1, x2])
        
        v -= lr * grad_v
        w -= lr * grad_w

    if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {total_loss / len(X):.4f}")

print("Final Weights (w2):\n", w)
print("Final Output Weights (w1):\n", v)
print(f"Final Loss: {total_loss / len(X):.4f}")

Epoch 0, Loss: 35934572.1467
Epoch 10, Loss: 35459142.4650
Epoch 20, Loss: 35459142.4650
Epoch 30, Loss: 35459142.4650
Epoch 40, Loss: 35459142.4650
Epoch 50, Loss: 35459142.4650
Epoch 60, Loss: 35459142.4650
Epoch 70, Loss: 35459142.4650
Epoch 80, Loss: 35459142.4650
Epoch 90, Loss: 35459142.4650
Final Weights (w):
 [[ 4.64995660e-01 -2.55294714e+00]
 [-4.07545960e+06 -1.03832159e+08]]
Final Output Weights (v):
 [-2.10217045e-01  1.21667919e+08]
Final Loss: 35459142.4650


# Q4

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [55]:
df = pd.read_csv("../HW3_train.csv")
df.fillna(df.median(), inplace=True)    # fill missing with median for now

X = df.drop(columns=["car_id", "price"])
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
# Model 1: Linear Regression with X1, X5
model1 = LinearRegression()
model1.fit(X_train[["X1", "X5"]], y_train)
y_pred1 = model1.predict(X_test[["X1", "X5"]])
loss1 = mean_squared_error(y_test, y_pred1)

# Model 2: Ridge Regression with X3, X9, X14
model2 = Ridge(alpha=1.0)
model2.fit(X_train[["X3", "X9", "X14"]], y_train)
y_pred2 = model2.predict(X_test[["X3", "X9", "X14"]])
loss2 = mean_squared_error(y_test, y_pred2)

# Model 3: Neural Network (Simple MLP)
model3 = MLPRegressor(hidden_layer_sizes=(10,), max_iter=500, random_state=42)
model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)
loss3 = mean_squared_error(y_test, y_pred3)

# Model 4: Neural Network (Deeper MLP)
model4 = MLPRegressor(hidden_layer_sizes=(50, 25), max_iter=1000, random_state=42)
model4.fit(X_train, y_train)
y_pred4 = model4.predict(X_test)
loss4 = mean_squared_error(y_test, y_pred4)

In [57]:
losses = {"Linear (X1, X5)": loss1, "Ridge (X3, X9, X14)": loss2, "MLP (10 nodes)": loss3, "MLP (50,25 nodes)": loss4}
best_model = min(losses, key=losses.get)
print("Best Model:", best_model)

Best Model: MLP (50,25 nodes)


In [60]:
# Load future dataset
future_df = pd.read_csv("../HW3_future.csv")
future_df.fillna(future_df.median(), inplace=True)

# Apply best model
if best_model == "Linear (X1, X5)":
    predictions = model1.predict(future_df[["X1", "X5"]])
elif best_model == "Ridge (X3, X9, X14)":
    predictions = model2.predict(future_df[["X3", "X9", "X14"]])
elif best_model == "MLP (10 nodes)":
    predictions = model3.predict(future_df.drop(columns=["car_id", 'price']))
elif best_model == "MLP (50,25 nodes)":
    predictions = model4.predict(future_df.drop(columns=["car_id", 'price']))

# Save predictions
output_df = pd.DataFrame({"car_id": future_df["car_id"], "price": predictions})
output_df.to_csv("../HW3_output.csv", index=False)